<a href="https://colab.research.google.com/github/vgaurav-umich/siads592/blob/master/analyse_covid_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import altair as alt
import numpy as np
import pandas as pd

In [0]:
from datetime import datetime as dt
from datetime import timedelta 

In [0]:
%run 'drive/My Drive/Colab Notebooks/get_covid19_data.ipynb'


==================================== Covid-19 NYT County Dataset =================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224666 entries, 0 to 224665
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    224666 non-null  datetime64[ns]
 1   county  224666 non-null  object        
 2   state   224666 non-null  object        
 3   fips    222381 non-null  float64       
 4   cases   224666 non-null  int64         
 5   deaths  224666 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 10.3+ MB


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (7,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,129,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,157,159,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,181,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,223,224,225,227,228,229,230,231,232,233,234,235,237,239,241,243,245,247,251,253,255,256,257,258,259,260,261,262,263,265,267,269,271,273,275,277,279,281,283,284,285,286,287,288,


============================== race_ethnicity_county_df: Cleaned Race Ethnicity dataset ===========================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3273 entries, 1 to 3273
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   fips                                       3272 non-null   float64
 1   geo_name                                   3273 non-null   object 
 2   hispanic_latino_any_race_PE                3273 non-null   float64
 3   non_hispanic_latino_any_race_PE            3273 non-null   float64
 4   white_alone_PE                             3273 non-null   float64
 5   black_african_american_alone_PE            3273 non-null   float64
 6   american_indian_alaska_native_alone_PE     3273 non-null   float64
 7   asian_alone_PE                             3273 non-null   float64
 8   native_hawaiian_pacific_islander_al

In [0]:
day_delay =  2 #@param {type:"integer"}

# Google colab runs on GMT timezone, so we will subtract 2 from curent date.
# If we get zero records, then please increase timedelta to 3 or 4. 
as_of = (dt.now() - timedelta(days = day_delay)).date()

my_green = '#86aa00'

## General Toll of Coronavirus

### COVID-19 by the Numbers

In [0]:
alt.Chart(
    covid19_demographic_state_df.query('date == @as_of'), 
    title = "Total Death Toll of COVID-19 in U.S."
).mark_text(
    color = my_green, 
    size = 50, 
    font = 'monospace', 
    dx = -280,
    align = "left"
).encode(
    text = alt.Text(
        'sum(deaths)',
        format = ',')
).configure_title(
    font = 'monospace', 
    anchor = 'start',
    dy = -10,
    fontSize = 25
).configure_view(
    stroke = 'transparent'
).properties(
    width = 600
) 

alt.Chart(...)

In [0]:
msa_cols_of_interest = ['date','cbsa', 'black_african_american_alone_PE',  'hispanic_latino_any_race_PE', 'cases', 'deaths', 'population', 'deaths_per_100k', 'cases_per_100k']           

test_df = covid19_demographic_cbsa_df.query('date == @as_of and deaths > 10')[msa_cols_of_interest]

cbsa_top10 = test_df.sort_values(['deaths_per_100k', 'cases_per_100k'], ascending = False).drop(['cases', 'population'], axis = 1).head(10)
alt.Chart(cbsa_top10, 
          title = 'Metro Area Top - 10'
    ).mark_bar(color = 'grey').encode(
        x = alt.X('deaths_per_100k:Q', title = '# of Deaths per 100 Thousand Population'), 
        y = alt.Y('cbsa', sort='-x', title = 'Metro Statistical Area'), 
        tooltip = ['deaths_per_100k', 'deaths', 'black_african_american_alone_PE', 'hispanic_latino_any_race_PE']).configure_title(
    font = 'monospace', 
    anchor = 'start',
    dy = -10,
    fontSize = 25
).configure_view(
    stroke = 'transparent'
).properties(
    width = 600
) 

alt.Chart(...)

In [0]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
source = covid19_demographic_state_df.query('date == @as_of')

alt.Chart(
    states, 
    title = "Infection Rate / 100 Thousand Population" 
).mark_geoshape(
    stroke = 'white'                                
).encode(
    color=alt.Color('cases_per_100k:Q', 
                    # scale=alt.Scale(range=['gray', my_green])
                    ), 
                    tooltip = [alt.Tooltip('state:N', title = 'State'),
                               alt.Tooltip('cases_per_100k:Q', 
                                           title = 'Case Rate/100K Population')
                               ],
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'fips', ['cases_per_100k', 'state'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
) .configure_title(
    fontSize = 25,
    align = 'left',  
    offset = 0, 
    anchor = 'start',
    dy = -10, 
    font = 'monospace', 
    subtitleFont = 'monospace', 
    subtitleFontSize = 18,
  ) 

alt.Chart(...)

In [0]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
source = covid19_demographic_state_df.query('date == @as_of')

alt.Chart(
    states, 
    title = "Death Rate / 100 Thousand Population" 
).mark_geoshape(
    stroke = 'white'                                
).encode(
    color=alt.Color('deaths_per_100k:Q', 
                    # scale=alt.Scale(range=['gray', 'red'])
                    ), 
                    tooltip = [alt.Tooltip('state:N', title = 'State'),
                               alt.Tooltip('deaths_per_100k:Q', 
                                           title = 'Death Rate/100K Population')
                               ],
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'fips', ['deaths_per_100k', 'state'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
) .configure_title(
    fontSize = 25,
    align = 'left',  
    offset = 0, 
    anchor = 'start',
    dy = -10, 
    font = 'monospace', 
    subtitleFont = 'monospace', 
    subtitleFontSize = 18,
  ) 

alt.Chart(...)

### We are witnessing a death rate of 2 to 10 %. 

i.e. every 2 to 10 person are dying after gettign infected. Of course, this depends on lot fo factors like how widespread testing efforts are in any given state and aging population. 

In [0]:
ratio_df = covid19_demographic_state_df.query(
    'date == @as_of'
    )[['state',
       'fips', 
       'cases', 
       'deaths',
       'population',
       'cases_per_100k',
       'deaths_per_100k']]

ratio_df['death_to_infection_ratio1'] = round(
    ratio_df['deaths_per_100k'] * 100 / ratio_df['cases_per_100k'],
    2
  )
  
ratio_df['death_to_infection_ratio2'] = round(
    ratio_df['deaths'] * 100 / ratio_df['cases'],
    2
  )


In [0]:

from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
source = ratio_df

alt.Chart(
    states, 
    title = "Death to Infection Cases Ratio" 
).mark_geoshape(
    stroke = 'white'                                
).encode(
    color=alt.Color('death_to_infection_ratio2:Q', 
                    # scale=alt.Scale(range=['gray', 'red'])
                    ), 
                    tooltip = [alt.Tooltip('state:N', title = 'State'),
                               alt.Tooltip('death_to_infection_ratio2:Q', 
                                           title = 'Death to Infection Cases Ratio')
                               ],
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'fips', ['death_to_infection_ratio2', 'state'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
) .configure_title(
    fontSize = 25,
    align = 'left',  
    offset = 0, 
    anchor = 'start',
    dy = -10, 
    font = 'monospace', 
    subtitleFont = 'monospace', 
    subtitleFontSize = 18,
  ) 

alt.Chart(...)

In [0]:
ratio_df = covid19_demographic_county_cbsa_df.query(
    'date == @as_of'
    )[['county',
       'fips', 
       'cases', 
       'deaths',
       'population',
       'cases_per_100k',
       'deaths_per_100k']]

from vega_datasets import data

counties = alt.topo_feature(data.us_10m.url, 'counties')
source = ratio_df

alt.Chart(
    counties, 
    title = "Death per 100K Population" 
).mark_geoshape(
    stroke = 'white'                                
).encode(
    color=alt.Color('deaths_per_100k:Q', 
                    # scale=alt.Scale(range=['gray', 'red'])
                    ), 
                    tooltip = [alt.Tooltip('county:N', title = 'County'),
                               alt.Tooltip('deaths_per_100k:Q', 
                                           title = 'Death to Infection Cases Ratio')
                               ],
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'fips', ['deaths_per_100k', 'county'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
) .configure_title(
    fontSize = 25,
    align = 'left',  
    offset = 0, 
    anchor = 'start',
    dy = -10, 
    font = 'monospace', 
    subtitleFont = 'monospace', 
    subtitleFontSize = 18,
  ) 

alt.Chart(...)

## **Finding #1** Black people represent nearly 23% of Deaths in United States, while they makeup only 13% of Population. 

In [0]:
# Let's prepare data for this visualization 
race_death_cols = ['Deaths_White',
        'Deaths_Black', 
        'Deaths_LatinX',
        'Deaths_Asian',
        'Deaths_AIAN', 
        'Deaths_NHPI', 
        'Deaths_Multiracial', 
        'Deaths_Other', 
        'Deaths_Unknown']

race_friendly_names = ['White', 'Black', 'Latino', 'Asian', 'American Indian', 'Native Hawaiin', 'Multiracial', 'Other', 'Unknown']

ethnicity_death_cols = ['Deaths_Ethnicity_Hispanic', 
        'Deaths_Ethnicity_NonHispanic', 
        'Deaths_Ethnicity_Unknown']

df = racial_analysis_df.query(
    'date == @as_of'
).melt(
    id_vars = ['date', 'state'],
    value_vars = race_death_cols, 
    var_name = 'death_count_type', 
    value_name = 'death_count'
)

df = df.groupby('death_count_type').sum().reset_index()
df['death_count_type'] = df['death_count_type'].str.replace('Deaths_', '')

In [0]:
bar = alt.Chart(df, title = "Total Death Toll of COVID-19 by Race").transform_joinaggregate(
    TotalDeath='sum(death_count)',
).transform_calculate(PercentOfTotal="datum.death_count / datum.TotalDeath").mark_bar().encode(
      x = alt.X('PercentOfTotal:Q', axis = alt.Axis(format = '.0%')),
      y = alt.Y('death_count_type:N', 
                title = None, sort = '-x'),
      color=alt.condition(
        alt.datum.death_count_type == 'Black',  # If the y axis value is Black this test returns True,
        alt.value(my_green),     # which sets the bar red.
        alt.value('grey'))   # And if it's not true it sets the bar .
  )

text = bar.mark_text(
    dx = 27, 
    font = 'monosapce', 
    fontSize = 15
  ).encode(
      text = alt.Text(
          'death_count:Q',
          format = ',')
  )

(bar + text).configure_title(font = 'monospace', 
                   anchor = 'start',
                   dy = -10,
                   fontSize = 25 ) \
  .configure_view(stroke = 'transparent') \
  .configure_axis(labelFont = 'monospace', labelFontSize = 15) \
  .properties(width = 600) 

alt.LayerChart(...)

In [0]:
# What is the percentage of Black Population in US?
# We have dataset race_ethnicity_county_df that contains total percent estimate for whole US. 
total_black_pop_prop = np.ceil(race_ethnicity_county_df.iloc[-1,:]['black_african_american_alone_PE'])
print(f"Total US Black Population proportion is - {total_black_pop_prop}")


Total US Black Population proportion is - 13.0


### YES, this is this significant!

Here we are going to conduct a simple one sample z-test of proportions.
We know that Blacks represent about 13% of US population, but we did observe based on data from 44 states that Black deaths are accounting for 23% of total deaths. 

In [0]:
from statsmodels.stats.proportion import proportions_ztest


ncount = df.query('death_count_type == "Black"')['death_count']
nobs = covid19_demographic_state_df.query('date == @as_of').sum()['deaths']
stat, pval = proportions_ztest(ncount, nobs, 0.23)
print(f"The z-score is {round(stat[0],2)}. \nThis means that what we observed represent an entirely new distribution of Deaths which is disproportional to US population.")

from decimal import Decimal
print('{0:.2E}'.format(Decimal(pval[0]))) 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The z-score is -16.35. 
This means that what we observed represent an entirely new distribution of Deaths which is disproportional to US population.
4.65E-60


##**Finding #2** Number of Infection cases in top 20 US counties are proportional to US population, but we see clear sign of disproportionate impact on people of color when we look at number of Deaths.

I did try to replicate analysis done by Covid Tracker project team  to infer county level impact of COVID-19 on poeple of color. 

State-level statistics tell part of the story, but many US states are also deeply segregated—meaning different counties in the same state can have vastly different breakdowns by race and ethnicity.

Race and ethnicity data for COVID cases isn't widely available at the county level, so we're using two numbers we do have: the latest infection and death rates for each county, from a New York Times dataset, paired with the largest racial or ethnic group in that county, based on the Census Bureau's 2018 ACS 5-Year estimates. The results are staggering.

In [0]:
def get_top20_counties_100k(case_type_death = True):
  col_name = 'deaths_per_100k' if case_type_death else 'cases_per_100k'
  print(col_name)

  # get top 20 counties with highest case type
  top20_county_cases_100k = covid19_demographic_county_cbsa_df.query('date == @as_of').sort_values(col_name, ascending = False).head(20)

  # Let's generate chart freindly name for county and column names
  top20_county_cases_100k['county_name'] = top20_county_cases_100k['county'] + ", " + top20_county_cases_100k['state_code']
  # we set index on county name becuase later we will be doing idmax
  top20_county_cases_100k = top20_county_cases_100k[['county_name',
                          'hispanic_latino_any_race_PE',	
                          'white_alone_PE',	
                          'black_african_american_alone_PE',
                          'american_indian_alaska_native_alone_PE',
                          col_name]].set_index(
                              'county_name'
                              ).rename(
                                  {'hispanic_latino_any_race_PE': 'Latino or Hispanic', 
                                  'white_alone_PE': 'White alone', 
                                  'black_african_american_alone_PE': 'Black or African American alone', 
                                  'american_indian_alaska_native_alone_PE': 'American Indian and Alaskan Native alone'}, 
                                  axis = 1
                              )
  # get the column name that contgains highest value 
  top20_county_cases_100k['largest_racial_group'] =  top20_county_cases_100k[['Latino or Hispanic',	
                          'White alone',	
                          'Black or African American alone',
                          'American Indian and Alaskan Native alone']].idxmax(axis = 1)
  # Finally, only select the columns we need.
  return top20_county_cases_100k.reset_index()[['county_name', col_name, 'largest_racial_group']]


In [0]:

color_scale = alt.Scale(
  domain = ['Latino or Hispanic',	
            'White alone',	
            'Black or African American alone',
            'American Indian and Alaskan Native alone'],
  range=["#f3a583",  "#cccccc" ,my_green,  "#94c6da" ])


In [0]:
# Dsiplay Infection rate Chart
bar = alt.Chart(
    get_top20_counties_100k(False), 
    title = "Counties with the 20 highest infection rates"
  ).mark_bar().encode(
      x = alt.X('cases_per_100k:Q', title = ''),
      y = alt.Y(
          'county_name:N', sort = '-x',  
          axis = alt.Axis(title = None, ticks = False)), 
      color = alt.Color(
          'largest_racial_group:N', 
          scale = color_scale, 
          legend = alt.Legend(
              title = 'Largest Racial Group'))
  )

(bar).configure_title(font = 'monospace', 
                   anchor = 'start',
                   dy = -10,
                   fontSize = 25 ) \
  .configure_view(stroke = 'transparent') \
  .configure_axis(labelFont = 'monospace', labelFontSize = 15) \
  .properties(width = 400) 

cases_per_100k


alt.Chart(...)

In [0]:
# Display Death rate Chart
bar = alt.Chart(
    get_top20_counties_100k(True), 
    title = "Counties with the 20 highest death rates"
  ).mark_bar().encode(
      x = alt.X('deaths_per_100k:Q', title = ''),
      y = alt.Y(
          'county_name:N', sort = '-x',  
          axis = alt.Axis(title = None, ticks = False)), 
      color = alt.Color(
          'largest_racial_group:N', 
          scale = color_scale, 
          legend = alt.Legend(
              title = 'Largest Racial Group'))
  )

(bar).configure_title(font = 'monospace', 
                   anchor = 'start',
                   dy = -10,
                   fontSize = 25 ) \
  .configure_view(stroke = 'transparent') \
  .configure_axis(labelFont = 'monospace', labelFontSize = 15) \
  .properties(width = 400) 

deaths_per_100k


alt.Chart(...)

## **Key Findings#3** We winessed huge disproportionality in Deaths

Here i tried to examine racial disproportionality in COVID-related mortality by comparing the percentage of COVID-19 deaths to the percentage of population. 

In [0]:
case_black_cols = ['date','state','fips','Cases_Total', 'cases', 'Cases_Black', 'Cases_Unknown', 'population', 'black_african_american_alone_PE'] 
death_black_cols = ['date','state','fips','Deaths_Total', 'deaths', 'Deaths_Black', 'Deaths_Unknown', 'population', 'black_african_american_alone_PE'] 

analysis_case_black_df = racial_analysis_df[case_black_cols]
analysis_death_black_df = racial_analysis_df[death_black_cols]
analysis_death_black_df['black_death_pct'] = round(analysis_death_black_df['Deaths_Black'] * 100 /analysis_death_black_df['Deaths_Total'])
analysis_death_black_df['pct_dff'] = round(analysis_death_black_df['black_death_pct'] - analysis_death_black_df['black_african_american_alone_PE'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
import altair as alt

df = analysis_death_black_df.query('Deaths_Total >= 20 and black_death_pct > 2').melt(id_vars = 'state', value_vars = ['black_african_american_alone_PE', 'black_death_pct'], var_name = 'percentage_type', value_name = 'percentage')
df['percentage'] = round(df['percentage'])

domain = ['black_african_american_alone_PE', 'black_death_pct']
range_ = ['gray', my_green]


title = alt.TitleParams(text = 'More Black Deaths, Relative To Their Population', 
                     subtitle = ['For all U.S. states with available data and Washington, D.C.,', 
                                 'in cases where 20 or more known deaths have occurred.', 
                                 'Sorted from most over-represented to most under-represented.'],
)

base = alt.Chart(df,  
                 title = title
                 ) \
        .encode(x = alt.X('percentage:Q', title = None, axis = None),
           y = alt.Y('state:N', sort = alt.EncodingSortField('percentage', op = 'variance', order = 'descending'),
                     axis = alt.Axis(
                        #  labelPadding = 120, 
                                     title = None,
                                     labelAlign = "right",
                                     ticks = True,
                                     grid = False, 
                                     tickBand = 'extent')))

point = base.mark_point(filled = True, 
                        size = 100, 
                        opacity = 1) \
          .encode(color = alt.Color('percentage_type:N', 
                                    legend = None,
                                    scale = alt.Scale(domain = domain,range = range_)))                                                                                  

text = base.mark_text(align='left',
    baseline='middle',
    dy= -10, 
    dx = 5,
    font = 'monospace'  
).encode(
    text= 'percentage:Q', detail = 'state:N'
)

line = base.mark_line(size = 8, 
                      opacity = 0.4) \
        .encode(detail = 'state:N', 
                color = alt.Color(value = 'gray'))

(line + point + text) \
  .configure_title(fontSize = 25,
                   align = 'left',  
                   offset = 0, 
                   anchor = 'start',
                   dy = -10, 
                   font = 'monospace', 
                   subtitleFont = 'monospace', 
                   subtitleFontSize = 18,
                   color = 'deepblue' ) \
  .configure_mark(font = 'monospace') \
  .configure_axis(labelFont = 'monospace',
                  labelFontSize = 15) \
  .configure_view(stroke = "transparent") \
  .properties(width = 600) 

alt.LayerChart(...)